In [84]:
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
import tensorflow
import json
import random
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Siddhant\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [110]:
def trainIntentModel():
    # Load the dataset and prepare it to the train the model

    # Importing dataset and splitting into words and labels
    dataset = pd.read_csv('./intent.csv', names=["Query", "Intent"])
    X = dataset["Query"]
    y = dataset["Intent"]
    unique_intent_list = list(set(y))
    print("Intent Dataset successfully loaded!")
    
    # Clean and prepare the intents corpus
    queryCorpus = []
    ps = PorterStemmer()
    for query in X:
        query = re.sub('[^a-zA-Z]', ' ', query)

        # Tokenize sentence
        query = query.split(' ')

        # Lemmatizing
        tokenized_query = [ps.stem(word.lower()) for word in query]
        print(tokenized_query)

        # Recreate the sentence from tokens
        tokenized_query = ' '.join(tokenized_query)
        print(tokenized_query)
        # Add to corpus
        queryCorpus.append(tokenized_query)
        
    print(queryCorpus)
    print("Corpus created")
    
    countVectorizer= CountVectorizer(max_features=800)
    corpus = countVectorizer.fit_transform(queryCorpus).toarray()
    print(corpus.shape)
    print(corpus)
    print("Bag of words created!")
    
    # Save the CountVectorizer
    pk.dump(countVectorizer, open("./IntentCountVectorizer.sav", 'wb'))
    print("Intent CountVectorizer saved!")
    
    # Encode the intent classes
    labelencoder_intent = LabelEncoder()
    y = labelencoder_intent.fit_transform(y)
    y = np_utils.to_categorical(y)
    print("Encoded the intent classes!")
    print(y)
    
    # Return a dictionary, mapping labels to their integer values
    res = {}
    for cl in labelencoder_intent.classes_:
        res.update({cl:labelencoder_intent.transform([cl])[0]})
    intent_label_map = res
    print(intent_label_map)
    print("Intent Label mapping obtained!")
    
    # Initialising the Aritifcial Neural Network
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 96, kernel_initializer = 'uniform', activation = 'relu', input_dim = 223))

    # Adding the second hidden layer
    classifier.add(Dense(units = 96, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Flatten())
    classifier.add(Dense(64, activation='relu'))
    classifier.add(Dense(53))
    # Adding the output layer
    classifier.add(Dense(units = 53, kernel_initializer = 'uniform', activation = 'softmax'))
   
    # Compiling the ANN
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    # Fitting the ANN to the Training set
    classifier.fit(corpus, y, batch_size = 10, epochs = 500)
    return classifier, intent_label_map

In [111]:
intent_model, intent_label_map = trainIntentModel()

Intent Dataset successfully loaded!
['queri']
queri
['hello', 'bot', '']
hello bot 
['hi']
hi
['hello', '']
hello 
['hey', 'there']
hey there
['hey']
hey
['namaskar']
namaskar
['namast']
namast
['bye']
bye
['see', 'you', 'later']
see you later
['goodby']
goodby
['thank']
thank
['thank', 'you']
thank you
['that', 's', 'help']
that s help
['thank', '', 'you', 'so', 'much']
thank  you so much
['what', 'hour', 'are', 'you', 'open', '']
what hour are you open 
['what', 'are', 'your', 'hour', '']
what are your hour 
['when', 'are', 'you', 'open', '']
when are you open 
['how', 'i', 'can', 'buy', 'crop', 'onlin', '']
how i can buy crop onlin 
['how', 'i', 'can', 'buy', 'crop', 'from', 'farmer', '']
how i can buy crop from farmer 
['how', 'can', 'i', 'buy', 'a', 'product', '']
how can i buy a product 
['how', 'i', 'can', 'sell', 'crop', 'onlin', '']
how i can sell crop onlin 
['how', 'i', 'can', 'sell', 'my', 'crop', '']
how i can sell my crop 
['best', 'way', 'to', 'sell', 'product', '']
best

23/23 [==============================] - 0s 2ms/step - loss: 3.9688 - accuracy: 0.0391
Epoch 2/500
23/23 [==============================] - 0s 1ms/step - loss: 3.9394 - accuracy: 0.0783
Epoch 3/500
23/23 [==============================] - 0s 1ms/step - loss: 3.7035 - accuracy: 0.1087
Epoch 4/500
23/23 [==============================] - 0s 1ms/step - loss: 3.1521 - accuracy: 0.1739
Epoch 5/500
23/23 [==============================] - 0s 1ms/step - loss: 2.5920 - accuracy: 0.2478
Epoch 6/500
23/23 [==============================] - 0s 1ms/step - loss: 2.1420 - accuracy: 0.3087
Epoch 7/500
23/23 [==============================] - 0s 1ms/step - loss: 1.7632 - accuracy: 0.4043
Epoch 8/500
23/23 [==============================] - 0s 1ms/step - loss: 1.4272 - accuracy: 0.4870
Epoch 9/500
23/23 [==============================] - 0s 1ms/step - loss: 1.1401 - accuracy: 0.5913
Epoch 10/500
23/23 [==============================] - 0s 1ms/step - loss: 0.9501 - accuracy: 0.6609
Epoch 11/500
23/23 [=

23/23 [==============================] - 0s 1ms/step - loss: 0.1618 - accuracy: 0.9043
Epoch 84/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1560 - accuracy: 0.9000
Epoch 85/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1561 - accuracy: 0.9087
Epoch 86/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1558 - accuracy: 0.8957
Epoch 87/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1621 - accuracy: 0.8957
Epoch 88/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1575 - accuracy: 0.9087
Epoch 89/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1563 - accuracy: 0.8957
Epoch 90/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1625 - accuracy: 0.9043
Epoch 91/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1574 - accuracy: 0.9000
Epoch 92/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1579 - accuracy: 0.9000
Epoch 93/500


23/23 [==============================] - 0s 2ms/step - loss: 0.1547 - accuracy: 0.9000
Epoch 165/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1532 - accuracy: 0.8913
Epoch 166/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1552 - accuracy: 0.9043
Epoch 167/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1584 - accuracy: 0.8783
Epoch 168/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1526 - accuracy: 0.9087
Epoch 169/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1566 - accuracy: 0.9000
Epoch 170/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1518 - accuracy: 0.9043
Epoch 171/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1554 - accuracy: 0.9043
Epoch 172/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1545 - accuracy: 0.8913
Epoch 173/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1609 - accuracy: 0.8739
Epoc

23/23 [==============================] - 0s 1ms/step - loss: 0.1528 - accuracy: 0.9000
Epoch 246/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1533 - accuracy: 0.9000
Epoch 247/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1496 - accuracy: 0.9043
Epoch 248/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1518 - accuracy: 0.8957
Epoch 249/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1507 - accuracy: 0.9087
Epoch 250/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1516 - accuracy: 0.9043
Epoch 251/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1556 - accuracy: 0.8957
Epoch 252/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1549 - accuracy: 0.9043
Epoch 253/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1518 - accuracy: 0.9000
Epoch 254/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1539 - accuracy: 0.9043
Epoc

23/23 [==============================] - 0s 1ms/step - loss: 0.1508 - accuracy: 0.9043
Epoch 327/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1529 - accuracy: 0.9087
Epoch 328/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1489 - accuracy: 0.9000
Epoch 329/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1545 - accuracy: 0.9043
Epoch 330/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1512 - accuracy: 0.9043
Epoch 331/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1514 - accuracy: 0.9000
Epoch 332/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1501 - accuracy: 0.9130
Epoch 333/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1496 - accuracy: 0.9043
Epoch 334/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1524 - accuracy: 0.9130
Epoch 335/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1548 - accuracy: 0.9043
Epoc

23/23 [==============================] - 0s 2ms/step - loss: 0.1559 - accuracy: 0.8957
Epoch 408/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1528 - accuracy: 0.9130
Epoch 409/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1495 - accuracy: 0.8913
Epoch 410/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1558 - accuracy: 0.9043
Epoch 411/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1505 - accuracy: 0.9043
Epoch 412/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1541 - accuracy: 0.9087
Epoch 413/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1493 - accuracy: 0.8913
Epoch 414/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1517 - accuracy: 0.9043
Epoch 415/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1505 - accuracy: 0.8870
Epoch 416/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1498 - accuracy: 0.9000
Epoc

23/23 [==============================] - 0s 2ms/step - loss: 0.1564 - accuracy: 0.8957
Epoch 489/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1524 - accuracy: 0.9000
Epoch 490/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1565 - accuracy: 0.9000
Epoch 491/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1545 - accuracy: 0.9043
Epoch 492/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1516 - accuracy: 0.9043
Epoch 493/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1551 - accuracy: 0.9087
Epoch 494/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1552 - accuracy: 0.8870
Epoch 495/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1572 - accuracy: 0.8957
Epoch 496/500
23/23 [==============================] - 0s 1ms/step - loss: 0.1533 - accuracy: 0.8957
Epoch 497/500
23/23 [==============================] - 0s 2ms/step - loss: 0.1558 - accuracy: 0.9130
Epoc

TypeError: predict() missing 1 required positional argument: 'x'

In [95]:
print(intent_label_map)

{'Cultivation': 0, 'Diseases': 1, 'Fertilizer': 2, 'Intent': 3, 'Pesticides': 4, 'askinghelp': 5, 'buy': 6, 'enquireaboutday': 7, 'family': 8, 'goodbye': 9, 'greeting': 10, 'groundnutbesttimeforsowing': 11, 'groundnutbestyield': 12, 'groundnutclimate': 13, 'groundnutcol': 14, 'groundnutcoldh256': 15, 'groundnutcondition': 16, 'groundnutdisease': 17, 'groundnuthowmuchseedstosow': 18, 'groundnutkharif': 19, 'groundnutkrishna': 20, 'groundnutnecrosiscontrol': 21, 'groundnutoil': 22, 'groundnutoil256': 23, 'groundnutplace': 24, 'groundnutplacekar': 25, 'groundnutrabi': 26, 'groundnutrodcontrol': 27, 'groundnutrustcontrol': 28, 'groundnutseedtreatment': 29, 'groundnutsuit': 30, 'groundnuttypes': 31, 'groundnutwater': 32, 'groundnutweather': 33, 'groundnutwhyrabi': 34, 'groundnuty101': 35, 'groundnuty24': 36, 'groundnuty256': 37, 'groundnuty257': 38, 'groundnutyeilddh101': 39, 'groundnutyeilddh256': 40, 'groundnutyeilddh257': 41, 'groundnutyeildtag24': 42, 'hours': 43, 'opentoday': 44, 'outo

Save the Intent model

In [96]:
intent_model.save('./intent_model.h5')
print("Intent model saved!")

Intent model saved!


In [97]:
def trainEntityModel():
    # Importing dataset and splitting into words and labels
    dataset = pd.read_csv('./data-tags.csv', names=["word", "label"])
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, 1].values
#     X = X.reshape(630,)
    print(X)
    print("Entity Dataset successfully loaded!")
    entityCorpus=[]
    ps = PorterStemmer()

    # Stem words in X
    for word in X.astype(str):
        word = [ps.stem(word[0])]
        entityCorpus.append(word)
    
    print(entityCorpus)
    X = entityCorpus
    from numpy import array
    X = array(X)
    X = X.reshape(len(X),)
    
    # Create a bag of words model for words
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer(max_features=1500)
#     X = cv.fit_transform(X.astype('U')).toarray()
    X = cv.fit_transform(X).toarray()
    print("Entity Bag of words created!")
    
    # Save CountVectorizer state
    pk.dump(cv, open('./EntityCountVectorizer.sav', 'wb'))
    print("Entity CountVectorizer state saved!")
    
    # Encoding categorical data of labels
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y.astype(str))
    print("Encoded the entity classes!")
    
    # Return a dict mapping labels to their integer values
    res = {}
    for cl in labelencoder_y.classes_:
        res.update({cl:labelencoder_y.transform([cl])[0]})
    entity_label_map = res
    print("Entity Label mapping obtained!")
    
    # Fit classifier to dataset
    classifier = GaussianNB()
    classifier.fit(X, y)
    print("Entity Model trained successfully!")
    
    # Save the entity classifier model
    pk.dump(classifier, open('./entity_model.sav', 'wb'))
    print("Trained entity model saved!")
    
    return entity_label_map

Load Entity model

In [98]:
entity_label_map = trainEntityModel()

[['word']
 ['how']
 ['to']
 ['grow']
 ['rice']
 ['?']
 ['jowar']
 ['how']
 ['much']
 ['water']
 ['to']
 ['add']
 ['for']
 ['wheat']
 ['?']
 ['i']
 ['best']
 ['season']
 ['to']
 ['grow']
 ['sugarcane']
 ['!']
 ['what']
 ['is']
 ['the']
 ['yield']
 ['of']
 ['rice']
 ['in']
 ['belgaum']
 ['?']
 ['best']
 ['pesticide']
 ['for']
 ['cotton']
 ['.']
 ['best']
 ['soil']
 ['for']
 ['cotton']
 ['.']
 ['how']
 ['much']
 ['fertilizer']
 ['should']
 ['i']
 ['add']
 ['to']
 ['my']
 ['rice']
 ['crop']
 ['?']
 ['when']
 ['to']
 ['sow']
 ['cotton']
 ['?']
 ['when']
 ['to']
 ['harvest']
 ['wheat']
 ['?']
 ['how']
 ['to']
 ['enhance']
 ['yield']
 ['of']
 ['sugarcane']
 ['?']
 ['i']
 ['want']
 ['to']
 ['grow']
 ['paddy']
 ['.']
 ['how']
 ['to']
 ['grow']
 ['maize']
 ['?']
 ['which']
 ['fertilizer']
 ['can']
 ['i']
 ['use']
 ['for']
 ['sugarcane']
 ['?']
 ['how']
 ['is']
 ['the']
 ['weather']
 ['today']
 ['?']
 ['how']
 ['is']
 ['the']
 ['weather']
 ['tomorrow']
 ['?']
 ['what']
 ['is']
 ['the']
 ['weather

In [99]:
loadedEntityCV = pk.load(open('./EntityCountVectorizer.sav', 'rb'))
loadedEntityClassifier = pk.load(open('./entity_model.sav', 'rb'))

In [100]:
def getEntities(query):
    query = loadedEntityCV.transform(query).toarray()
    
    response_tags = loadedEntityClassifier.predict(query)
    
    entity_list=[]
    for tag in response_tags:
        if tag in entity_label_map.values():
            entity_list.append(list(entity_label_map.keys())[list(entity_label_map.values()).index(tag)])
    return entity_list

In [101]:
with open('./intents.json') as json_data:
    intents = json.load(json_data)

In [102]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

Load model to predict user result

In [103]:
loadedIntentClassifier = load_model('./intent_model.h5')
loaded_intent_CV = pk.load(open('./IntentCountVectorizer.sav', 'rb'))    

In [104]:
USER_INTENT = ""

In [105]:
# while True:
#     user_query = input()
    
#     query = re.sub('[^a-zA-Z]', ' ', user_query)

#     # Tokenize sentence
#     query = query.split(' ')

#     # Lemmatizing
#     ps = PorterStemmer()
#     tokenized_query = [ps.stem(word.lower()) for word in query]

#     # Recreate the sentence from tokens
#     processed_text = ' '.join(tokenized_query)
    
#     # Transform the query using the CountVectorizer
#     processed_text = loaded_intent_CV.transform([processed_text]).toarray()

#     # Make the prediction
#     predicted_Intent = loadedIntentClassifier.predict(processed_text)
# #     print(predicted_Intent)
#     result = np.argmax(predicted_Intent, axis=1)
    
#     for key, value in intent_label_map.items():
#         if value==result[0]:
#             #print(key)
#             USER_INTENT = key
#             break
#         # Extract entities from text
#     entities = getEntities(tokenized_query)
    
#     # Mapping between tokens and entity tags
#     token_entity_map = dict(zip(entities, tokenized_query))
#     print(token_entity_map)
    
#     for i in intents['intents']:
#         if i['tag'] == USER_INTENT:
#             print(random.choice(i['responses']))

In [106]:
user_query = input()
    
query = re.sub('[^a-zA-Z]', ' ', user_query)

    # Tokenize sentence
query = query.split(' ')

    # Lemmatizing
ps = PorterStemmer()
tokenized_query = [ps.stem(word.lower()) for word in query]

    # Recreate the sentence from tokens
processed_text = ' '.join(tokenized_query)
    
    # Transform the query using the CountVectorizer
processed_text = loaded_intent_CV.transform([processed_text]).toarray()

    # Make the prediction
predicted_Intent = loadedIntentClassifier.predict(processed_text)
print(predicted_Intent)
result = np.argmax(predicted_Intent, axis=1)
# print(result)
for key, value in intent_label_map.items():
    if value==result[0]:
        print(key)
        USER_INTENT = key
        break
        
for i in intents['intents']:
    if i['tag'] == USER_INTENT:
        print(random.choice(i['responses']))
            
    # Extract entities from text
entities = getEntities(tokenized_query)
    
    # Mapping between tokens and entity tags
token_entity_map = dict(zip(entities, tokenized_query))
print(token_entity_map)

hi
1/1 [==============================] - 0s 44ms/step
[[1.8178250e-27 5.2060016e-24 4.5699718e-14 1.9440942e-09 6.1176070e-22
  2.9477702e-15 1.9992983e-16 4.1739466e-07 8.6110849e-06 3.9943807e-06
  9.9996769e-01 3.5719791e-21 2.6775220e-23 8.1198427e-19 3.5253554e-15
  1.6308284e-15 7.5574030e-23 6.0774163e-21 7.6963761e-06 2.3673463e-21
  1.6976267e-13 2.5872854e-15 8.9685159e-07 2.0817721e-07 3.6097600e-20
  5.1634006e-16 7.8830087e-17 4.5388086e-12 1.6663369e-19 1.6393120e-14
  2.6696797e-15 8.1815715e-24 2.7450239e-24 1.9015216e-19 5.7856060e-15
  4.2516271e-23 2.5001768e-29 6.6558119e-23 5.3737609e-23 2.0980070e-15
  2.0797276e-15 1.9473764e-15 6.7616457e-19 2.2480522e-08 3.0377786e-14
  7.9900565e-07 5.2478413e-06 4.4935650e-06 2.1959840e-16 6.0072034e-15
  1.9309393e-08 9.8310349e-10 3.6943070e-12]]
greeting
Hello
{'GREET': 'hi'}
